In [ ]:
import cv2
import face_recognition
import pytesseract
import numpy as np
import os
from threading import Thread, Lock
from queue import Queue
import time
import platform
import re
import csv
from datetime import datetime
import pandas as pd

# ========== OPTIMIZED CONFIGURATION ==========
CONFIG = {
    "FACE_MATCH_THRESHOLD": 0.6,
    "PLATE_OCR_CONFIG": r'--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789',
    "MIN_PLATE_CHARS": 3,
    "FRAME_SKIP": 5,  # Increased from 3 to 5
    "OCR_CONFIDENCE_THRESHOLD": 40.0,
    "DETECTION_COOLDOWN": 2.0,  # Seconds between processing the same plate
    "CSV_PATHS": {
        "OFFENDER_PLATES": "csvs/offender_license_plates.csv",
        "ALL_PLATES": "csvs/all_license_plates.csv"
    },
    "CAMERAS": [
        {
            "name": "Mobile",
            "src": "http://192.168.29.231:8080/video",
            "role": "plates",
            "width": 640,  # Reduced from 1280 for better performance
            "height": 360,  # Reduced from 720 for better performance
            "fps": 15
        },
        {
            "name": "Laptop",
            "src": 0,
            "role": "faces",
            "width": 480,  # Reduced from 640 for better performance
            "height": 360,  # Reduced from 480
            "fps": 20
        }
    ],
    # Choose only one pipeline variant for plates
    "PLATE_PIPELINE": "adaptive"  # Options: "adaptive", "edge", "morph", "bilateral"
}

# ========== INITIALIZATION ==========
print("🔍 Loading reference faces...")
try:
    known_face_encodings = []
    known_face_names = []
    
    reference_folder = "reference_images"
    os.makedirs(reference_folder, exist_ok=True)
    reference_image_path = os.path.join(reference_folder, "person.jpg")
    
    if not os.path.exists(reference_image_path):
        raise FileNotFoundError(f"Reference image not found at {reference_image_path}")
    
    ref_image = face_recognition.load_image_file(reference_image_path)
    ref_encodings = face_recognition.face_encodings(ref_image)
    
    if not ref_encodings:
        raise ValueError("No faces found in reference image")
    
    known_face_encodings.append(ref_encodings[0])
    known_face_names.append("Authorized Person")
    print(f"✅ {len(known_face_encodings)} reference faces loaded")
except Exception as e:
    print(f"❌ Face load error: {e}")
    exit()

# Tesseract setup
if platform.system() == 'Windows':
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Create CSV directory and files if they don't exist
os.makedirs("csvs", exist_ok=True)

# Initialize CSV file for all license plates if it doesn't exist
if not os.path.exists(CONFIG["CSV_PATHS"]["ALL_PLATES"]):
    with open(CONFIG["CSV_PATHS"]["ALL_PLATES"], 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['timestamp', 'license_plate', 'confidence', 'is_offender'])
    print(f"✅ Created {CONFIG['CSV_PATHS']['ALL_PLATES']}")

# Load offender license plates
offender_plates = set()
try:
    if os.path.exists(CONFIG["CSV_PATHS"]["OFFENDER_PLATES"]):
        offender_df = pd.read_csv(CONFIG["CSV_PATHS"]["OFFENDER_PLATES"])
        if 'license_plate' in offender_df.columns:
            offender_plates = set(plate.strip().upper() for plate in offender_df['license_plate'].astype(str) if plate.strip())
            print(f"✅ Loaded {len(offender_plates)} offender license plates")
        else:
            print(f"⚠️ No 'license_plate' column found in {CONFIG['CSV_PATHS']['OFFENDER_PLATES']}")
    else:
        print(f"⚠️ Offender plates file not found at {CONFIG['CSV_PATHS']['OFFENDER_PLATES']}")
        # Create an empty file with header
        with open(CONFIG["CSV_PATHS"]["OFFENDER_PLATES"], 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['license_plate'])
        print(f"✅ Created empty offender plates file")
except Exception as e:
    print(f"❌ Error loading offender plates: {e}")
    offender_plates = set()

# Statistics tracking
stats = {
    "plates_matched": 0,
    "plates_current_frame": 0,
    "faces_detected": 0,
    "faces_matched": 0,
    "plates_detected_total": 0,
    "last_detected_plates": {},  # Changed to dict to track times: {plate: timestamp}
    "face_detection_last_run": 0,  # Track when we last ran face detection
    "face_resize_factor": 0.5,  # Start with 0.5, adjust dynamically
    "frame_counter": 0,
    "fps_history": []
}

# ========== OPTIMIZED VIDEO STREAM ==========
class VideoStream:
    def __init__(self, src, name="Camera", width=640, height=480, fps=30):
        self.stream = cv2.VideoCapture(src)
        if not self.stream.isOpened():
            raise RuntimeError(f"Cannot open {name} camera")
        
        self.stream.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.stream.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        self.stream.set(cv2.CAP_PROP_FPS, fps)
        self.stream.set(cv2.CAP_PROP_BUFFERSIZE, 2)
        self.name = name
        self.role = "faces" if name == "Laptop" else "plates"
        self.frame_queue = Queue(maxsize=2)
        self.lock = Lock()
        self.running = False
        self.width = width
        self.height = height
        self.last_read_time = time.time()
        
    def start(self):
        self.running = True
        self.thread = Thread(target=self.update, daemon=True)
        self.thread.start()
        return self
    
    def update(self):
        while self.running:
            ret, frame = self.stream.read()
            if ret:
                # Only resize if needed
                if frame.shape[1] != self.width or frame.shape[0] != self.height:
                    frame = cv2.resize(frame, (self.width, self.height))
                
                with self.lock:
                    if self.frame_queue.full():
                        self.frame_queue.get()
                    self.frame_queue.put(frame)
            time.sleep(0.02)  # Reduced update frequency
    
    def read(self):
        with self.lock:
            if self.frame_queue.empty():
                return None
            frame = self.frame_queue.get()
            self.last_read_time = time.time()
            return frame
        
    def stop(self):
        self.running = False
        if hasattr(self, 'thread'):
            self.thread.join()
        self.stream.release()

# ========== CSV HANDLING ==========
def save_license_plate(plate_text, confidence=None):
    """Save a license plate to the CSV file with optional confidence score"""
    # Check if we've seen this plate recently
    current_time = time.time()
    if plate_text in stats["last_detected_plates"]:
        # Only process if it's been more than the cooldown period
        if current_time - stats["last_detected_plates"][plate_text] < CONFIG["DETECTION_COOLDOWN"]:
            return  # Skip this detection, too soon
    
    # Update the timestamp for this plate
    stats["last_detected_plates"][plate_text] = current_time
    
    # Save the plate data
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    is_offender = "Yes" if plate_text in offender_plates else "No"
    
    try:
        with open(CONFIG["CSV_PATHS"]["ALL_PLATES"], 'a', newline='') as file:
            writer = csv.writer(file)
            if confidence is not None:
                # Include confidence score and offender status
                writer.writerow([timestamp, plate_text, f"{confidence:.1f}", is_offender])
            else:
                # Backward compatibility
                writer.writerow([timestamp, plate_text, "N/A", is_offender])
                
        stats["plates_detected_total"] += 1
    except Exception as e:
        print(f"⚠️ Error saving plate to CSV: {e}")

# ========== PROCESSING FUNCTIONS ==========
def detect_faces(frame):
    if frame is None:
        return None
    
    try:
        # Implement time-based throttling for face detection
        current_time = time.time()
        if current_time - stats["face_detection_last_run"] < 0.1:  # Run at max 10 fps
            return frame  # Skip face detection this frame
        
        stats["face_detection_last_run"] = current_time
        
        # Reset face counters for this frame
        stats["faces_detected"] = 0
        stats["faces_matched"] = 0
        
        # Convert to RGB (face_recognition uses RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Dynamic scaling based on performance
        scale = stats["face_resize_factor"]
        small_frame = cv2.resize(rgb_frame, (0, 0), fx=scale, fy=scale)
        
        # Find face locations using HOG (faster than CNN)
        face_locations = face_recognition.face_locations(small_frame, model="hog")
        
        # Adjust scale factor based on processing time
        process_time = time.time() - current_time
        if process_time > 0.1:  # If processing takes > 100ms
            stats["face_resize_factor"] = max(0.25, stats["face_resize_factor"] - 0.05)  # Reduce scale
        elif process_time < 0.05 and stats["face_resize_factor"] < 0.5:  # If processing is fast
            stats["face_resize_factor"] = min(0.5, stats["face_resize_factor"] + 0.05)  # Increase scale
        
        # Only compute encodings if we have faces (saves computation)
        if face_locations:
            face_encodings = face_recognition.face_encodings(small_frame, face_locations)
        else:
            face_encodings = []
        
        # Update stats
        stats["faces_detected"] = len(face_locations)
        
        # Draw debug info
        cv2.putText(frame, f"Faces: {stats['faces_detected']}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Scale back up face locations
            top = int(top / scale)
            right = int(right / scale)
            bottom = int(bottom / scale)
            left = int(left / scale)
            
            matches = face_recognition.compare_faces(
                known_face_encodings, 
                face_encoding, 
                tolerance=CONFIG["FACE_MATCH_THRESHOLD"]
            )
            
            if True in matches:
                # Known face - green box
                color = (0, 0, 255)
                name = "Got Em"
                stats["faces_matched"] += 1
            else:
                # Unknown face - gray box
                color = (80, 80, 80)
                name = "Unidentified"
            
            # Draw bounding box and label
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Add match stats
        cv2.putText(frame, f"Matches: {stats['faces_matched']}/{stats['faces_detected']}", 
                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        return frame
    except Exception as e:
        print(f"⚠️ Face detection error: {e}")
        return frame

def preprocess_plate_image(plate_img):
    """Apply a single, optimized preprocessing pipeline to plate image"""
    # Choose one preprocessing pipeline based on configuration
    pipeline = CONFIG.get("PLATE_PIPELINE", "adaptive")
    
    if pipeline == "edge":
        # Edge-based preprocessing
        blurred = cv2.GaussianBlur(plate_img, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        return edges
    
    elif pipeline == "morph":
        # Morphological preprocessing
        _, thresh = cv2.threshold(plate_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        return cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    elif pipeline == "bilateral":
        # Bilateral filtering
        bilateral = cv2.bilateralFilter(plate_img, 11, 17, 17)
        _, thresh = cv2.threshold(bilateral, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return thresh
    
    else:  # Default to adaptive
        # Adaptive thresholding (most reliable)
        return cv2.adaptiveThreshold(plate_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY_INV, 11, 2)

def detect_plates(frame):
    if frame is None:
        return None
    
    try:
        # Only process every Nth frame based on FRAME_SKIP
        stats["frame_counter"] += 1
        if stats["frame_counter"] % CONFIG["FRAME_SKIP"] != 0:
            # Display current stats without processing
            cv2.putText(frame, f"Plates: {stats['plates_current_frame']}", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            cv2.putText(frame, f"Offenders: {stats['plates_matched']}", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"Total: {stats['plates_detected_total']}", 
                       (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            return frame
        
        # Reset plate counter for this frame
        stats["plates_current_frame"] = 0
        detected_plates = []
        
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply CLAHE for better contrast
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        gray = clahe.apply(gray)
        
        # Choose a single edge detection method (faster)
        # Using Canny for edge detection (faster than combined Sobel)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        
        # Find contours
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter contours by area and sort by size
        contours = [c for c in contours if 500 < cv2.contourArea(c) < 50000]
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]  # Reduced candidates for speed
        
        # Debug info
        cv2.putText(frame, f"Contours: {len(contours)}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        for contour in contours:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
            
            # Check if contour is roughly quadrilateral
            if 4 <= len(approx) <= 8:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)
                
                # License plate aspect ratio range
                if 1.5 < aspect_ratio < 7.0:
                    # Extract the plate region
                    plate_img = gray[max(0, y-5):min(frame.shape[0], y+h+5), 
                                    max(0, x-5):min(frame.shape[1], x+w+5)]
                    
                    # Check plate size before OCR
                    if plate_img.shape[0] > 15 and plate_img.shape[1] > 15:
                        # Resize for better OCR - ensure minimum width
                        scale_factor = max(1.5, 150 / plate_img.shape[1])
                        plate_img = cv2.resize(plate_img, (0, 0), fx=scale_factor, fy=scale_factor)
                        
                        # Apply a single optimized preprocessing pipeline
                        processed_plate = preprocess_plate_image(plate_img)
                        
                        # Try OCR with just one or two configs instead of 12 different combinations
                        config = CONFIG["PLATE_OCR_CONFIG"]
                        try:
                            # Run OCR once with the optimized config
                            result = pytesseract.image_to_data(processed_plate, config=config, 
                                                            output_type=pytesseract.Output.DICT)
                            
                            # Extract text and confidence
                            confidences = [int(conf) for conf in result['conf'] if conf != '-1']
                            
                            if confidences:  # If we got any valid text
                                avg_confidence = sum(confidences) / len(confidences)
                                text = pytesseract.image_to_string(processed_plate, config=config)
                                clean_text = re.sub(r'[^A-Z0-9]', '', text.upper())
                                
                                # Only process if confidence is good and text length is reasonable
                                if (len(clean_text) >= CONFIG["MIN_PLATE_CHARS"] and 
                                    avg_confidence >= CONFIG["OCR_CONFIDENCE_THRESHOLD"]):
                                    
                                    stats["plates_current_frame"] += 1
                                    detected_plates.append(clean_text)
                                    
                                    # Add confidence to display
                                    confidence_text = f"{avg_confidence:.1f}% conf"
                                    
                                    # Add plate overlay
                                    if clean_text in offender_plates:
                                        # Red box for offender plate
                                        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
                                        cv2.putText(frame, f"OFFENDER: {clean_text}", 
                                                (x, y-25), cv2.FONT_HERSHEY_SIMPLEX, 
                                                0.7, (0, 0, 255), 2)
                                        cv2.putText(frame, confidence_text, 
                                                (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 
                                                0.5, (0, 0, 255), 1)
                                        stats["plates_matched"] += 1
                                    else:
                                        # Blue box for normal plate
                                        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                                        cv2.putText(frame, f"PLATE: {clean_text}", 
                                                (x, y-25), cv2.FONT_HERSHEY_SIMPLEX, 
                                                0.7, (255, 0, 0), 2)
                                        cv2.putText(frame, confidence_text, 
                                                (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 
                                                0.5, (255, 0, 0), 1)
                                    
                                    # Save plate to CSV with confidence score
                                    save_license_plate(clean_text, avg_confidence)
                        except Exception as e:
                            # Just continue with next contour if OCR fails
                            continue
        
        # Display stats on frame
        cv2.putText(frame, f"Plates this frame: {stats['plates_current_frame']}", 
                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Offender matches: {stats['plates_matched']}", 
                   (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"Total plates: {stats['plates_detected_total']}", 
                   (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        # Prune old detections from the cache
        current_time = time.time()
        stats["last_detected_plates"] = {
            plate: timestamp for plate, timestamp in stats["last_detected_plates"].items() 
            if current_time - timestamp < CONFIG["DETECTION_COOLDOWN"] * 3
        }
        
        return frame
    except Exception as e:
        print(f"⚠️ Plate detection error: {e}")
        return frame

# ========== MAIN EXECUTION ==========
def main():
    print("📷 Starting camera streams...")
    cameras = []
    for cam in CONFIG["CAMERAS"]:
        try:
            camera = VideoStream(
                cam["src"],
                cam["name"],
                cam["width"],
                cam["height"],
                cam.get("fps", 15)
            ).start()
            cameras.append(camera)
            print(f"  ✅ {cam['name']} camera initialized for {cam['role']} - {cam['width']}x{cam['height']}")
        except Exception as e:
            print(f"  ❌ Failed to initialize {cam['name']} camera: {e}")
            cameras.append(None)
    
    try:
        # Create windows
        for cam in CONFIG["CAMERAS"]:
            cv2.namedWindow(cam["name"] + " Feed", cv2.WINDOW_NORMAL)
            cv2.resizeWindow(cam["name"] + " Feed", cam["width"], cam["height"])
        
        print("🚀 Starting surveillance system (Press Q to quit)...")
        last_time = time.time()
        fps_display_time = time.time()
        
        while True:
            loop_start_time = time.time()
            
            for camera in cameras:
                if camera is None:
                    continue
                
                frame = camera.read()
                if frame is None:
                    continue
                
                # Role-based processing
                if camera.role == "faces":
                    processed_frame = detect_faces(frame)
                else:
                    processed_frame = detect_plates(frame)
                
                if processed_frame is not None:
                    # Calculate and display FPS (but update display less frequently)
                    current_time = time.time()
                    fps = 1 / (current_time - last_time) if (current_time - last_time) > 0 else 0
                    last_time = current_time
                    
                    # Add to FPS history
                    stats["fps_history"].append(fps)
                    if len(stats["fps_history"]) > 30:
                        stats["fps_history"].pop(0)
                    avg_fps = sum(stats["fps_history"]) / len(stats["fps_history"])
                    
                    # Update FPS display only every half second to reduce CPU usage
                    if current_time - fps_display_time > 0.5:
                        fps_display_time = current_time
                        fps_text = f"{processed_frame.shape[1]}x{processed_frame.shape[0]} | FPS: {avg_fps:.1f}"
                        
                        # Create a black background for FPS text
                        cv2.rectangle(processed_frame, 
                                    (5, processed_frame.shape[0] - 30), 
                                    (250, processed_frame.shape[0] - 5), 
                                    (0, 0, 0), -1)
                        
                        cv2.putText(processed_frame, fps_text, 
                                   (10, processed_frame.shape[0] - 10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    cv2.imshow(camera.name + " Feed", processed_frame)
            
            # Exit on Q key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            # Intelligent frame rate control
            elapsed = time.time() - loop_start_time
            target_delay = 1/25  # Target ~25 FPS overall
            if elapsed < target_delay:
                time.sleep(target_delay - elapsed)
    
    except KeyboardInterrupt:
        print("\n🛑 Received interrupt signal")
    except Exception as e:
        print(f"🔥 Critical error: {e}")
    finally:
        print("🛑 Stopping system...")
        for camera in cameras:
            if camera is not None:
                camera.stop()
        cv2.destroyAllWindows()
        print("✅ System stopped cleanly")
        print(f"📊 Final Stats:")
        print(f"  - Total plates detected: {stats['plates_detected_total']}")
        print(f"  - Offender plates matched: {stats['plates_matched']}")
        print(f"  - CSV saved to: {CONFIG['CSV_PATHS']['ALL_PLATES']}")

if __name__ == "__main__":
    main()

🔍 Loading reference faces...
✅ 1 reference faces loaded
✅ Loaded 2 offender license plates
📷 Starting camera streams...
  ✅ Mobile camera initialized for plates - 640x360
  ✅ Laptop camera initialized for faces - 480x360
🚀 Starting surveillance system (Press Q to quit)...
🛑 Stopping system...
✅ System stopped cleanly
📊 Final Stats:
  - Total plates detected: 0
  - Offender plates matched: 0
  - CSV saved to: csvs/all_license_plates.csv
  - OCR Method Success Rates:


In [ ]:
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import json
import asyncio
import cv2
import base64
import numpy as np
import face_recognition
import pytesseract
import os
import re
import time
import platform
from typing import List, Dict, Any, Optional
from datetime import datetime

# Initialize FastAPI app
app = FastAPI(title="Surveillance System ML Service")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, replace with specific origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ========== CONFIGURATION ==========
CONFIG = {
    "FACE_MATCH_THRESHOLD": 0.6,
    "PLATE_OCR_CONFIG": r'--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789',
    "MIN_PLATE_CHARS": 3,
    "FRAME_SKIP": 3,  # Process plates every N frames
}

# Tesseract setup
if platform.system() == 'Windows':
    pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# Database simulation
plates_db = []
faces_db = []

# ========== FACE RECOGNITION SETUP ==========
print("Loading reference faces...")
try:
    known_face_encodings = []
    known_face_names = []
    
    reference_folder = "reference_images"
    os.makedirs(reference_folder, exist_ok=True)
    reference_image_path = os.path.join(reference_folder, "person.jpg")
    
    if os.path.exists(reference_image_path):
        ref_image = face_recognition.load_image_file(reference_image_path)
        ref_encodings = face_recognition.face_encodings(ref_image)
        
        if ref_encodings:
            known_face_encodings.append(ref_encodings[0])
            known_face_names.append("Authorized Person")
            print(f"{len(known_face_encodings)} reference faces loaded")
        else:
            print("No faces found in reference image")
    else:
        print(f"Reference image not found at {reference_image_path}")
        print(f"Please add a reference image at {os.path.abspath(reference_image_path)}")
except Exception as e:
    print(f"Face load error: {e}")

# ========== PROCESSING FUNCTIONS ==========
def detect_faces(frame):
    if frame is None:
        return None, []
    
    try:
        # Convert to RGB (face_recognition uses RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Scale down for faster processing
        scale = 0.5
        small_frame = cv2.resize(rgb_frame, (0, 0), fx=scale, fy=scale)
        
        # Find face locations using HOG (faster than CNN)
        face_locations = face_recognition.face_locations(small_frame, model="hog")
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)
        
        detected_faces = []
        
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Scale back up face locations
            top = int(top / scale)
            right = int(right / scale)
            bottom = int(bottom / scale)
            left = int(left / scale)
            
            matches = face_recognition.compare_faces(
                known_face_encodings, 
                face_encoding, 
                tolerance=CONFIG["FACE_MATCH_THRESHOLD"]
            )
            
            if True in matches:
                # Known face
                name = "Authorized Person"
                status = "authorized"
            else:
                # Unknown face
                name = "Unidentified"
                status = "unknown"
            
            # Draw bounding box and label
            color = (0, 0, 255) if status == "authorized" else (80, 80, 80)
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            detected_faces.append({
                "name": name,
                "status": status,
                "location": {
                    "top": top,
                    "right": right,
                    "bottom": bottom,
                    "left": left
                },
                "timestamp": datetime.now().isoformat()
            })
            
            # Add to database
            faces_db.append({
                "id": len(faces_db) + 1,
                "name": name,
                "status": status,
                "timestamp": datetime.now().isoformat()
            })
        
        return frame, detected_faces
    except Exception as e:
        print(f"Face detection error: {e}")
        return frame, []

def detect_plates(frame, frame_counter):
    if frame is None:
        return None, []
    
    # Skip frames to reduce processing load
    if frame_counter % CONFIG["FRAME_SKIP"] != 0:
        return frame, []
    
    try:
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        gray = clahe.apply(gray)
        
        # Apply Gaussian blur to reduce noise
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY_INV, 11, 2)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter contours by area
        contours = [c for c in contours if cv2.contourArea(c) > 1000]
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]  # Top 5 candidates
        
        detected_plates = []
        
        for contour in contours:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
            
            # More relaxed shape condition - 4-sided or more
            if 4 <= len(approx) <= 6:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)

                # License plate aspect ratio check
                if 2.0 < aspect_ratio < 6.0:
                    plate_img = gray[y:y+h, x:x+w]

                    
                    # Display potential license plate region
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
                    
                    # Check plate size before OCR
                    if plate_img.shape[0] > 0 and plate_img.shape[1] > 0:
                        # Apply additional preprocessing to the plate region
                        plate_img = cv2.resize(plate_img, (0, 0), fx=2, fy=2)  # Upscale
                        plate_img = cv2.GaussianBlur(plate_img, (3, 3), 0)
                        _, plate_img = cv2.threshold(plate_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
                        
                        # OCR with Tesseract
                        text = pytesseract.image_to_string(
                            plate_img,
                            config=CONFIG["PLATE_OCR_CONFIG"]
                        )
                        clean_text = re.sub(r'[^A-Z0-9]', '', text.upper())
                        
                        if len(clean_text) >= CONFIG["MIN_PLATE_CHARS"]:
                            # Simulate offender check (in real app, check against database)
                            is_offender = len(clean_text) % 2 == 0  # Just for demo
                            
                            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                            cv2.putText(frame, f"PLATE: {clean_text}", 
                                       (x, y-15), cv2.FONT_HERSHEY_SIMPLEX, 
                                       0.7, (255, 0, 0), 2)
                            
                            plate_data = {
                                "plateNumber": clean_text,
                                "isOffender": is_offender,
                                "timestamp": datetime.now().isoformat(),
                                "location": {"x": x, "y": y, "width": w, "height": h}
                            }
                            
                            detected_plates.append(plate_data)
                            
                            # Add to database
                            plates_db.append({
                                "id": len(plates_db) + 1,
                                "plateNumber": clean_text,
                                "isOffender": is_offender,
                                "timestamp": datetime.now().isoformat(),
                                "cameraId": "CAM-1207"  # Example camera ID
                            })
        
        return frame, detected_plates
    except Exception as e:
        print(f"Plate detection error: {e}")
        return frame, []

# ========== WEBSOCKET MANAGER ==========
class ConnectionManager:
    def __init__(self):
        self.active_connections: List[WebSocket] = []

    async def connect(self, websocket: WebSocket):
        await websocket.accept()
        self.active_connections.append(websocket)

    def disconnect(self, websocket: WebSocket):
        self.active_connections.remove(websocket)

    async def send_personal_message(self, message: str, websocket: WebSocket):
        await websocket.send_text(message)

    async def broadcast(self, message: str):
        for connection in self.active_connections:
            await connection.send_text(message)

manager = ConnectionManager()

# ========== API ROUTES ==========
@app.get("/")
async def root():
    return {"message": "Surveillance System ML Service is running"}

@app.get("/stats")
async def get_stats():
    total_records = len(plates_db) + len(faces_db)
    offenders_today = sum(1 for plate in plates_db if plate.get("isOffender", False))
    
    last_alert = None
    if plates_db:
        offender_plates = [p for p in plates_db if p.get("isOffender", False)]
        if offender_plates:
            last_alert = offender_plates[-1]
    
    return {
        "totalRecords": total_records,
        "offendersToday": offenders_today,
        "camerasActive": 2,  # Hardcoded for demo
        "lastAlert": last_alert
    }

@app.get("/plates")
async def get_plates():
    return plates_db

@app.get("/faces")
async def get_faces():
    return faces_db

# ========== WEBSOCKET ENDPOINT ==========
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await manager.connect(websocket)
    frame_counter = 0
    
    try:
        while True:
            data = await websocket.receive_text()
            data = json.loads(data)
            
            if data["type"] == "frame":
                # Decode base64 image
                img_data = base64.b64decode(data["frame"].split(',')[1])
                nparr = np.frombuffer(img_data, np.uint8)
                frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                
                # Process based on camera role
                if data["role"] == "faces":
                    processed_frame, detected_faces = detect_faces(frame)
                    if detected_faces:
                        await websocket.send_json({
                            "type": "faces",
                            "data": detected_faces
                        })
                else:  # plates
                    processed_frame, detected_plates = detect_plates(frame, frame_counter)
                    if detected_plates:
                        await websocket.send_json({
                            "type": "plates",
                            "data": detected_plates
                        })
                
                # Encode processed frame to base64
                if processed_frame is not None:
                    _, buffer = cv2.imencode('.jpg', processed_frame)
                    processed_frame_b64 = base64.b64encode(buffer).decode('utf-8')
                    
                    await websocket.send_json({
                        "type": "processed_frame",
                        "frame": f"data:image/jpeg;base64,{processed_frame_b64}",
                        "camera": data["camera"]
                    })
                
                frame_counter += 1
    
    except WebSocketDisconnect:
        manager.disconnect(websocket)
    except Exception as e:
        print(f"WebSocket error: {e}")
        manager.disconnect(websocket)

# ========== SERVER STARTUP ==========
if __name__ == "__main__":
    print("Starting ML Service on http://localhost:8000")
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)
